# 7DaysOfCode - Machine Learning 

## Day-2: Data Pre-processing

Pre-processing is a crucial step in the Machine Learning workflow as it ensures the quality of our data. This step involves cleaning, organizing, and transforming raw data into high-quality data that can be used to train models. Properly preprocessed data directly impacts the ability of our models to learn and make accurate predictions.

Some of the data preprocessing steps, such as handling null values and duplicates, were already addressed earlier in this documentation. In this step, I will focus on enriching the dataset, standardization, and checking for multicollinearity. 

In [66]:
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from yellowbrick.model_selection import FeatureImportances
pd.set_option('display.max_columns', None)

In [2]:
spotify_df = pd.read_csv("C:/Users/SAMSUNG/OneDrive/Documentos/GitHub/7DaysOfCode_SpotifyML/data/day_one.csv")

In [3]:
spotify_df.shape

(89740, 20)

## Creating the binary class

As I am dealing with a binary classification problem, where the goal is to predict whether a song will be popular or not, I need to encode the 'popularity' column as the target variable. I have chosen a threshold of 70, which means that songs with a popularity score above 70 will be considered popular and encoded as 1, while those with a score below this threshold will be considered not popular and encoded as 0.

In [4]:
#target value: popularity
spotify_df['popularity'] = spotify_df['popularity'].apply(lambda x: 1 if x > 70 else 0)

## Feature engineering
Feature engineering is the process of creating new featurs or transforming existing features in a dataset to improve the perfomance of machine learning models. In this step I will remove irrelevant columns and create new features.


In [5]:
spotify_df.head()

track_id                 artists  \
0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...   
4                                            Hold On   

                   track_name  popularity  duration_ms  explicit  \
0                      Comedy           1     3.844433     False   
1            Ghost - Acoustic           0     2.493500     False   
2              To Begin Again           0     3.513767     False   
3  Can't Help Falling In Love           1     3.365550     False   
4                     Hold On           1     3.314217     False   

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.676  0.4610    1    -6.746     0       0.1430        0.0322   
1         0.420  0.1660    1   -17.235     1       0.0763        0.9240   
2         0.438  0.3590    0    -9.734     1       0.0557        0.2100   
3         0.266  0.0596    0   -18.515     1       0.0363        0.9050   
4         0.618  0.4430    2    -9.681     1       0.0526        0.4690   

   instrumentalness  liveness  valence    tempo  time_signature track_genre  
0          0.000001    0.3580    0.715   87.917               4    acoustic  
1          0.000006    0.1010    0.267   77.489               4    acoustic  
2          0.000000    0.1170    0.120   76.332               4    acoustic  
3          0.000071    0.1320    0.143  181.740               3    acoustic  
4          0.000000    0.0829    0.167  119.949               4    acoustic

In this initial stage,  I will be working exclusively with numeric data in this dataset. I have chosen to exclude categorical variables for some reasons:

i. Limited contribution to model performance: Variables such as track_id, album_name, and track_name are unlikely to significantly improve the performance of the model, as they may not contain meaningful numerical information for the model to learn from.

ii. High cardinality of variables: Variables like genre and artist may have a large number of unique categories, which can result in increased computational overhead and additional data cleaning steps. Considering the potential gains in model performance, dealing with these variables may be time-consuming and cumbersome.

By excluding these categorical variables, I can streamline the data preparation process and focus on working with numerical data, which may be more relevant for the specific modeling task at hand.

In [6]:
spotify_df = spotify_df.drop(columns = ['track_id', 'album_name', 'track_name', 'artists', 'track_genre'])

In [7]:
spotify_df.head(10)

popularity  duration_ms  explicit  danceability  energy  key  loudness  \
0           1     3.844433     False         0.676  0.4610    1    -6.746   
1           0     2.493500     False         0.420  0.1660    1   -17.235   
2           0     3.513767     False         0.438  0.3590    0    -9.734   
3           1     3.365550     False         0.266  0.0596    0   -18.515   
4           1     3.314217     False         0.618  0.4430    2    -9.681   
5           0     3.570667     False         0.688  0.4810    6    -8.807   
6           1     3.823333     False         0.407  0.1470    2    -8.822   
7           1     4.049100     False         0.703  0.4440   11    -9.331   
8           1     3.160217     False         0.625  0.4140    0    -8.700   
9           0     3.426567     False         0.442  0.6320    1    -6.770   

   mode  speechiness  acousticness  instrumentalness  liveness  valence  \
0     0       0.1430        0.0322          0.000001    0.3580   0.7150   
1     1       0.0763        0.9240          0.000006    0.1010   0.2670   
2     1       0.0557        0.2100          0.000000    0.1170   0.1200   
3     1       0.0363        0.9050          0.000071    0.1320   0.1430   
4     1       0.0526        0.4690          0.000000    0.0829   0.1670   
5     1       0.1050        0.2890          0.000000    0.1890   0.6660   
6     1       0.0355        0.8570          0.000003    0.0913   0.0765   
7     1       0.0417        0.5590          0.000000    0.0973   0.7120   
8     1       0.0369        0.2940          0.000000    0.1510   0.6690   
9     1       0.0295        0.4260          0.004190    0.0735   0.1960   

     tempo  time_signature  
0   87.917               4  
1   77.489               4  
2   76.332               4  
3  181.740               3  
4  119.949               4  
5   98.017               4  
6  141.284               3  
7  150.960               4  
8  130.088               4  
9   78.899               4

### One Hot enconding

Handling categorical data is a critical step in the machine learning workflow. Even though I have excluded categorical data with high cardinality, there are still some remaining variables that need to be addressed. Certain machine learning models are unable to work with categorical data directly, and it's important to avoid misinterpretation of categorical variables as continuous ones, such as interpreting key as an integer variable when it is actually a categorical variable. To address this, I will be utilizing One Hot Encoding to appropriately represent these categorical variables in the dataset.

In [8]:
def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix, drop_first=True) # Drop the first column to avoid multicollinearity
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [9]:
#getting the dumies
spotify_df = onehot_encode(spotify_df, 'mode', 'mode')
spotify_df = onehot_encode(spotify_df, 'explicit', 'isExplicit')
spotify_df = onehot_encode(spotify_df, 'key', 'key')
spotify_df = onehot_encode(spotify_df, 'time_signature', 'time_signature')

In [10]:
spotify_df.shape

(89740, 28)

In [11]:
spotify_df.head()

popularity  duration_ms  danceability  energy  loudness  speechiness  \
0           1     3.844433         0.676  0.4610    -6.746       0.1430   
1           0     2.493500         0.420  0.1660   -17.235       0.0763   
2           0     3.513767         0.438  0.3590    -9.734       0.0557   
3           1     3.365550         0.266  0.0596   -18.515       0.0363   
4           1     3.314217         0.618  0.4430    -9.681       0.0526   

   acousticness  instrumentalness  liveness  valence    tempo  mode_1  \
0        0.0322          0.000001    0.3580    0.715   87.917       0   
1        0.9240          0.000006    0.1010    0.267   77.489       1   
2        0.2100          0.000000    0.1170    0.120   76.332       1   
3        0.9050          0.000071    0.1320    0.143  181.740       1   
4        0.4690          0.000000    0.0829    0.167  119.949       1   

   isExplicit_True  key_1  key_2  key_3  key_4  key_5  key_6  key_7  key_8  \
0                0      1      0      0      0      0      0      0      0   
1                0      1      0      0      0      0      0      0      0   
2                0      0      0      0      0      0      0      0      0   
3                0      0      0      0      0      0      0      0      0   
4                0      0      1      0      0      0      0      0      0   

   key_9  key_10  key_11  time_signature_1  time_signature_3  \
0      0       0       0                 0                 0   
1      0       0       0                 0                 0   
2      0       0       0                 0                 0   
3      0       0       0                 0                 1   
4      0       0       0                 0                 0   

   time_signature_4  time_signature_5  
0                 1                 0  
1                 1                 0  
2                 1                 0  
3                 0                 0  
4                 1                 0


## Scaling

In this step, I will perform standardization. Standardization involves transforming the values of the dataset such that the mean of the values is 0 and the standard deviation is 1. This is crucial because many machine learning models, including those that use the gradient descent algorithm, are sensitive to the scale of the input features. Having columns with different scales can result in issues such as overshooting or loss of computational efficiency. Standardization helps to mitigate these problems and ensures that the data is normalized to a common scale, making it more suitable for machine learning model training and inference.

In [12]:
spotify_df.columns

Index(['popularity', 'duration_ms', 'danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'mode_1', 'isExplicit_True', 'key_1', 'key_2',
       'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10',
       'key_11', 'time_signature_1', 'time_signature_3', 'time_signature_4',
       'time_signature_5'],
      dtype='object')

In [13]:
y = spotify_df.loc[:, 'popularity']
X = spotify_df.drop('popularity', axis = 1)

In [14]:
#standardization
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

## Day-3: Train Test Validation

### Train-test sets

Today, my focus is on splitting the dataset into train, test, and validation sets. In machine learning, we split the data into train and test sets to allow our model to learn from the patterns in the training data. Once the learning is done, the model can then make predictions on unseen data from the test set. This helps us evaluate how well the model can generalize to new, unseen data and assess its performance.This step is important to avoid issues with overfitting or underfitting, which can negatively impact the model's ability to generalize to new data and result in lower accuracy.

A common practice is to allocate around 80% of the data for training and 20% for testing. This is the approach I will be using to split the dataset.

In [15]:
print(len(X_std))
print(len(y))

89740
89740


In [16]:
y.value_counts(normalize=True)

0    0.969545
1    0.030455
Name: popularity, dtype: float64

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2, random_state = 42, shuffle=True)

### Cross-Validation

One potential drawback of simply splitting the dataset into train/test sets is that the splitting may not be random, which can lead to issues with overfitting. To address this concern, I will be performing a cross-validation step. Cross-validation involves creating multiple train/test sets by splitting the data into k subsets, and training the model on k-1 of these subsets. There are various cross-validation methods available, and for this exercise, I will be using the StratifiedKFold method.

The `StratifiedKFold` method is particularly useful as it ensures that the class proportions within each fold are as close as possible to the real class proportions in the overall dataset. By using cross-validation, we can obtain a more robust assessment of our model's performance and ensure that it can generalize well to unseen data, improving its reliability for real-world scenarios.

In [18]:
# Create an instance of StratifiedKFold with the desired number of splits
skf = StratifiedKFold(n_splits= 5, shuffle= True, random_state= 42)

# Iterate over the splits and perform cross-validation
for train_index, val_index in skf.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train.reset_index(drop=True)[train_index], y_train.reset_index(drop=True)[val_index]  ## .reset_index is to ensure data alignment

**Note:** The `.reset_index`method in pandas is used to ensure proper alignment of data. It is typically used to reset the index of a dataframe, which can be useful in cases where the index is not sequential or needs to be re-aligned after performing certain operations on the data. This step can help avoid issues such as bugs caused by misaligned data (which I encountered in my code). By using.`reset_index`, we can ensure that the data is in the correct order and alignment, which is essential for accurate analysis and modeling.

## D-4: Modeling and Training

### Baselines: Dummy Classifier and Logistic Regression

#### Dummy Classifier

Now that I have split the data, I will be working on some baseline models. A baseline model serves as a starting point for performance comparison with other models. It is usually a simple model that allows us to establish a benchmark. If a model performs better than the baseline model, then it can be considered a good model. Additionally, a baseline model can be useful in identifying potential issues with bias or variance. For example, if a baseline model performs poorly, with low accuracy, it may indicate problems with bias or variance in the data.

I will start with a **dummy classifier**. A dummy classifier is so named because it does not use the data to make predictions. Instead, it uses a pre-defined strategy that we pass to it. In this case, I will set the strategy to 'most frequent', which predicts the most frequent class in the dataset. In our dataset, the most frequent class is '0', as there are more music items that fall under the label of 'not popular' (scoring less than 70 on the popularity score) than those that are popular.

The dummy classifier provides a null accuracy baseline, which is the accuracy that can be achieved by always predicting the most frequent class. While a dummy classifier is not typically used in real-world classification problems, it can be useful in certain scenarios. If the models I'm going to train perform close to the performance of the dummy classifier, it may indicate that the features in the model are ineffective, erroneously computed, or missing for some reason. It could also indicate a large class imbalance, where the accuracy gains on the test set simply applied to too few examples to produce a significant gain. In such cases, I will need to work on solutions for the imbalanced class and consider other metrics besides accuracy. Therefore, dummy classifiers can provide a useful sanity check and point of comparison for evaluating model performance.

In [23]:

dummy_model = DummyClassifier(strategy = 'most_frequent' )

#training the model
dummy_model.fit(X_train_fold, y_train_fold)

#predicting on the train data
dummy_pred_train = dummy_model.predict(X_train_fold)

#predicting on the validation data
dummy_pred_val = dummy_model.predict(X_val_fold)



In [38]:
print(f'Prediction on dummy classifier set on train data \n', dummy_pred_train[:200])
print(f'Prediction on dummy classifier set on validation data \n', dummy_pred_val[:200])



Prediction on dummy classifier set on train data 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Prediction on dummy classifier set on validation data 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


Above you can read the result from DummyClassifier's predictions. It's always predicting 0 or the negative class for every instance in the test set.  Now we can call the usual score method to get the accuracy of the DummyClassifier's constant negative prediction. 

In [41]:
print(f'accuracy on the training set: ',dummy_model.score(X_train_fold, y_train_fold))
print(f'accuracy on the validation set: ',dummy_model.score(X_val_fold, y_val_fold))

accuracy on the training set:  0.9691820176202249
accuracy on the validation set:  0.9691461206296141


An accuracy score of 0.96 is very high, but it should be noted that this dataset is imbalanced, so such a high accuracy is expected.

### Logistic Regression

Logistic regression can be seen as a kind of generalized linear model. Like ordinary least squares and other regression methods, logistic regression takes a set of input variables (features) and estimates a target value. However, unlike ordinary linear regression, the target value in logistic regression is binary, typically represented as 0 or 1. The logistic regression model uses a logistic function to transform the input from a real value to an output Y that is bounded between 0 and 1. This output is interpreted as the probability of the object belonging to the positive class, given the input characteristics. 

In [43]:
#regression model
log_model = LogisticRegression()

# training the model
log_model.fit(X_train_fold, y_train_fold)

# predicting on the train set
log_pred_train = log_model.predict(X_train_fold)

# predicting on the validation set
log_pred_val = log_model.predict(X_val_fold)

In [78]:
log_odds = np.exp(log_model.coef_[0])

In [79]:
pd.DataFrame(log_odds,
             X.columns,
             columns=['coef']).sort_values(by='coef', ascending = False)

coef
loudness          1.879194
danceability      1.232885
isExplicit_True   1.211023
key_8             1.090023
key_1             1.074752
key_11            1.051540
key_9             1.033019
key_3             1.030284
key_5             1.026017
key_10            1.024640
key_4             1.022513
key_6             1.020652
valence           0.987762
key_2             0.984284
mode_1            0.972700
key_7             0.959818
duration_ms       0.952894
tempo             0.939759
speechiness       0.804861
time_signature_5  0.783366
time_signature_1  0.758144
liveness          0.758096
time_signature_3  0.714035
time_signature_4  0.684397
acousticness      0.665290
energy            0.598361
instrumentalness  0.450752

Logistic Regression models can be more challenging to interpret due to the use of the logit function, which returns probabilities. To make sense of these probabilities, we often convert them to odds by taking the exponential of the logs. For example, when the `loudness` variable increases by one unit, the odds of the music being in the target class ("1") are over 1.8 times as large as the odds of it not being in the target class. On the other hand, when the `instrumentalness` variable increases by one unit, the odds of the music being in the target class are only around 0.4. Since the odds are less than one, we can take the reciprocal (1/odds) to gain better insight. Thus, as instrumentalness increases by 1, the odds of the music **NOT** being in the target class are 2.2 times as likely as the odds of it being in the target class.

In [44]:
# Calculate accuracy on the train set
accuracy_train = log_model.score(X_train_fold, y_train_fold)
print("Accuracy on train set:", accuracy_train)

# Calculate accuracy on the validation set
accuracy_val = log_model.score(X_val_fold, y_val_fold)
print("Accuracy on validation set:", accuracy_val)


Accuracy on train set: 0.9691820176202249
Accuracy on validation set: 0.9691461206296141


Upon calculating only the accuracy, we can observe that the Logistic Regression model, as it stands, is not performing better than the Dummy Classifier. This suggests that we may have issues with an imbalanced dataset. To improve the generalization of the model, we will need to address this issue by applying methods such as resampling techniques to mitigate the impact of class imbalance.